In [ ]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *
how_many=128
# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=5
classes=10
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:how_many]

label=np.full((how_many),9)
for i in range(classes-1):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    label_temp=np.full((how_many),i)
    

    all=np.concatenate((x[:how_many],all))
    label=np.concatenate((label_temp,label))
   
random_index = np.random.permutation(how_many*classes)
all=torch.tensor(all).to(device)
label=torch.tensor(label).to(device)
# Mischiare i campioni e le etichette utilizzando l'indice di permutazione casuale

#data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

qc_array=np.array([124,126])
min_array=np.array([0.1,0.2])
beta_array=np.array([1e-2,5e-2,1e-3,5e-3])
layer_array=np.array([75]) 
num_batch=classes
print(NUM_QUBITS)
print(T)
print(beta_array)

for n_layer in layer_array:
    for qc in qc_array:
        for beta_indx in beta_array:
            for min_b in min_array:
                betas      = np.insert(np.linspace(beta_indx,min_b, T), 0, 0)
                
                alphas     = 1 - betas
                alphas_bar = np.cumprod(alphas)
                
                betas      = torch.tensor(betas).float().to(device)
                alphas     = torch.tensor(alphas).float().to(device)
                alphas_bar = torch.tensor(alphas_bar).float().to(device)
                theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
                optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
                scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
                trained_thetas_1 = []
                loss_history = []
                best_loss = 1e10

                for epoch in range(NUM_EPOCHS):
                    print(epoch)

                    t0 = time.time()
                    num_batch=classes
                    tot_loss=0
                    random_index = np.random.permutation(how_many*classes)

                    # Mischiare i campioni e le etichette utilizzando l'indice di permutazione casuale
                    mnist_images = all[random_index]
                    labels = label[random_index]

                    for batch_indx in range(int(how_many*classes/BATCH_SIZE)): #qua dovrebbe essere il numero di batch, ma visto che sono uguali al numero di classi metto il numero di clasii
                        loss_batch=0
                        image_batch=mnist_images[BATCH_SIZE*batch_indx:BATCH_SIZE*(batch_indx+1)]
                        label_batch=labels[BATCH_SIZE*batch_indx:BATCH_SIZE*(batch_indx+1)]


                        t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                        betas_batch = betas[t].to(device)
                        alphas_batch=alphas_bar[t].to(device)

                        # assemble input at t add noise (t+1)
                        target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)


                        input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                        target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                        input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                        for j in range(classes):

                            indici_classe = (label_batch == j).nonzero().squeeze()

                            img_batch_temp=input_batch[indici_classe]
                            target_batch_temp=input_batch[indici_classe]

                            zero_input = torch.zeros((len(img_batch_temp), 2**NUM_QUBITS), dtype=torch.complex64)
                            zero_target = torch.zeros((len(img_batch_temp), 2**NUM_QUBITS), dtype=torch.complex64)


                            zero_input[:,j*ld_dim:(j+1)*ld_dim]=img_batch_temp

                            zero_target[:,j*ld_dim:(j+1)*ld_dim]=target_batch_temp


                            loss_label = loss_fn_aq(qc,theta_1,n_layer, zero_input, zero_target,j)
                            #if j==9: print(f'loss{loss_label}, label={j}')
                            loss_batch+=loss_label/classes



                        tot_loss+=loss_batch.item()/num_batch
                        loss_batch.backward()
                        optimizer.step()
                        optimizer.zero_grad()





                    trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                    loss_history.append(tot_loss)
                    if tot_loss< best_loss:
                        best_loss=tot_loss

                    # implement learning rate scheduler
                    scheduler.step()


                # print every epoch
                    print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {tot_loss:.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} beta {beta_indx} nl{n_layer} QC{qc}')
                    #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')

                np.save(f'thetas/all_thetas_T{T}_nl{n_layer}_min{min_b}_beta{beta_indx}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}_cond_all.npy',trained_thetas_1)
                np.save(f'losses/all_loss__T{T}_nl{n_layer}_min{min_b}_beta{beta_indx}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}_cond_all.npy',loss_history)

7
5
[0.01  0.05  0.001 0.005]
0
T=5 Epoch: 1/25 - Loss: 0.3214 b_loss=0.3214 - T: 165.47s/epoch ,tempo_previto=273.02 min0.1 beta 0.01 nl75 QC124
1
T=5 Epoch: 2/25 - Loss: 0.1519 b_loss=0.1519 - T: 151.50s/epoch ,tempo_previto=247.46 min0.1 beta 0.01 nl75 QC124
2
T=5 Epoch: 3/25 - Loss: 0.1188 b_loss=0.1188 - T: 143.67s/epoch ,tempo_previto=232.27 min0.1 beta 0.01 nl75 QC124
3
T=5 Epoch: 4/25 - Loss: 0.1066 b_loss=0.1066 - T: 155.73s/epoch ,tempo_previto=249.16 min0.1 beta 0.01 nl75 QC124
4
T=5 Epoch: 5/25 - Loss: 0.0979 b_loss=0.0979 - T: 197.92s/epoch ,tempo_previto=313.37 min0.1 beta 0.01 nl75 QC124
5
T=5 Epoch: 6/25 - Loss: 0.0892 b_loss=0.0892 - T: 184.91s/epoch ,tempo_previto=289.70 min0.1 beta 0.01 nl75 QC124
6
T=5 Epoch: 7/25 - Loss: 0.0846 b_loss=0.0846 - T: 192.30s/epoch ,tempo_previto=298.07 min0.1 beta 0.01 nl75 QC124
7
T=5 Epoch: 8/25 - Loss: 0.0815 b_loss=0.0815 - T: 190.71s/epoch ,tempo_previto=292.42 min0.1 beta 0.01 nl75 QC124
8
T=5 Epoch: 9/25 - Loss: 0.0788 b_loss=0.

T=5 Epoch: 21/25 - Loss: 0.0909 b_loss=0.0902 - T: 158.99s/epoch ,tempo_previto=209.33 min0.1 beta 0.05 nl75 QC124
21
T=5 Epoch: 22/25 - Loss: 0.0953 b_loss=0.0902 - T: 165.16s/epoch ,tempo_previto=214.71 min0.1 beta 0.05 nl75 QC124
22
T=5 Epoch: 23/25 - Loss: 0.0922 b_loss=0.0902 - T: 159.14s/epoch ,tempo_previto=204.23 min0.1 beta 0.05 nl75 QC124
23
T=5 Epoch: 24/25 - Loss: 0.0929 b_loss=0.0902 - T: 155.95s/epoch ,tempo_previto=197.54 min0.1 beta 0.05 nl75 QC124
24
T=5 Epoch: 25/25 - Loss: 0.0932 b_loss=0.0902 - T: 162.41s/epoch ,tempo_previto=203.02 min0.1 beta 0.05 nl75 QC124
0
T=5 Epoch: 1/25 - Loss: 0.3479 b_loss=0.3479 - T: 165.14s/epoch ,tempo_previto=272.48 min0.2 beta 0.05 nl75 QC124
1
T=5 Epoch: 2/25 - Loss: 0.2212 b_loss=0.2212 - T: 146.83s/epoch ,tempo_previto=239.82 min0.2 beta 0.05 nl75 QC124
2
T=5 Epoch: 3/25 - Loss: 0.1908 b_loss=0.1908 - T: 159.33s/epoch ,tempo_previto=257.59 min0.2 beta 0.05 nl75 QC124
3
T=5 Epoch: 4/25 - Loss: 0.1708 b_loss=0.1708 - T: 156.37s/epoch

T=5 Epoch: 16/25 - Loss: 0.0948 b_loss=0.0939 - T: 160.27s/epoch ,tempo_previto=224.37 min0.2 beta 0.001 nl75 QC124
16
T=5 Epoch: 17/25 - Loss: 0.0972 b_loss=0.0939 - T: 173.15s/epoch ,tempo_previto=239.52 min0.2 beta 0.001 nl75 QC124
17
T=5 Epoch: 18/25 - Loss: 0.0963 b_loss=0.0939 - T: 159.25s/epoch ,tempo_previto=217.64 min0.2 beta 0.001 nl75 QC124
18
T=5 Epoch: 19/25 - Loss: 0.0969 b_loss=0.0939 - T: 161.90s/epoch ,tempo_previto=218.56 min0.2 beta 0.001 nl75 QC124
19
T=5 Epoch: 20/25 - Loss: 0.0942 b_loss=0.0939 - T: 157.87s/epoch ,tempo_previto=210.49 min0.2 beta 0.001 nl75 QC124
20
T=5 Epoch: 21/25 - Loss: 0.0890 b_loss=0.0890 - T: 169.87s/epoch ,tempo_previto=223.67 min0.2 beta 0.001 nl75 QC124
21
T=5 Epoch: 22/25 - Loss: 0.0970 b_loss=0.0890 - T: 159.62s/epoch ,tempo_previto=207.51 min0.2 beta 0.001 nl75 QC124
22
T=5 Epoch: 23/25 - Loss: 0.0966 b_loss=0.0890 - T: 163.91s/epoch ,tempo_previto=210.35 min0.2 beta 0.001 nl75 QC124
23
T=5 Epoch: 24/25 - Loss: 0.0969 b_loss=0.0890 - 

T=5 Epoch: 11/25 - Loss: 0.0993 b_loss=0.0993 - T: 170.27s/epoch ,tempo_previto=252.56 min0.1 beta 0.01 nl75 QC126
11
T=5 Epoch: 12/25 - Loss: 0.1009 b_loss=0.0993 - T: 173.48s/epoch ,tempo_previto=254.44 min0.1 beta 0.01 nl75 QC126
12
T=5 Epoch: 13/25 - Loss: 0.0957 b_loss=0.0957 - T: 166.88s/epoch ,tempo_previto=241.97 min0.1 beta 0.01 nl75 QC126
13
T=5 Epoch: 14/25 - Loss: 0.0944 b_loss=0.0944 - T: 164.11s/epoch ,tempo_previto=235.23 min0.1 beta 0.01 nl75 QC126
14
T=5 Epoch: 15/25 - Loss: 0.0962 b_loss=0.0944 - T: 157.80s/epoch ,tempo_previto=223.54 min0.1 beta 0.01 nl75 QC126
15
T=5 Epoch: 16/25 - Loss: 0.0978 b_loss=0.0944 - T: 165.73s/epoch ,tempo_previto=232.02 min0.1 beta 0.01 nl75 QC126
16
T=5 Epoch: 17/25 - Loss: 0.0959 b_loss=0.0944 - T: 144.17s/epoch ,tempo_previto=199.43 min0.1 beta 0.01 nl75 QC126
17
T=5 Epoch: 18/25 - Loss: 0.0979 b_loss=0.0944 - T: 164.62s/epoch ,tempo_previto=224.98 min0.1 beta 0.01 nl75 QC126
18
T=5 Epoch: 19/25 - Loss: 0.0941 b_loss=0.0941 - T: 150.7